In [1]:
import os
import requests
import json

curr_dir = os.getcwd()

def get_file_path(fname): 
    return os.path.join(curr_dir, fname)

In [2]:
snake_names_file = 'snakes.txt'
try:
    with open(get_file_path(snake_names_file), 'r') as file:
        snake_names = file.readlines()
        snake_names = [name.rstrip() for name in snake_names]
        print(snake_names)
except FileNotFoundError:
    print("Error: snakes.txt not found, run nc_snakes notebook")


['Carphophis amoenus', 'Cemophora coccinea', 'Coluber constrictor', 'Diadophis punctatus', 'Pantherophis guttatus', 'Pantherophis obsoletus', 'Farancia abacura', 'Farancia erytrogramma', 'Heterodon platirhinos', 'Heterodon simus', 'Lampropeltis rhombomaculata', 'Lampropeltis getula', 'Lampropeltis elapsoides', 'Lampropeltis triangulum triangulum', 'Masticophis flagellum', 'Nerodia erythrogaster', 'Nerodia fasciata', 'Nerodia sipedon', 'Nerodia taxispilota', 'Opheodrys aestivus', 'Pituophis melanoleucus', 'Regina rigida', 'Regina septemvitatta', 'Rhadinaea flavilata', 'Seminatrix pygaea', 'Storeria dekayi', 'Storeria occipitomaculata', 'Tantilla coronata', 'Thamnophis sauritus', 'Thamnophis sirtalis', 'Haldea striatula', 'Virginia valeriae', 'Micrurus fulvius', 'Agkistrodon contortrix', 'Agkistrodon piscivorus', 'Crotalus adamanteus', 'Crotalus horridus', 'Sistrurus miliarius']


In [3]:
snake_ids_file = 'snake_ids.json'

api_base_url = "https://api.inaturalist.org/v1"
get_observations_url = api_base_url + "/observations"

def search_species_url(s_name):
    return f"{api_base_url}/search?q={s_name}&sources=taxa"


In [4]:
if os.path.exists(get_file_path(snake_ids_file)):
    print("Snake ids file already exists, skipping...")
    snake_ids = json.load(open(get_file_path(snake_ids_file), 'r'))
else:
    # Get map of snake names to species ids
    snake_ids = {}

    for snake_name in snake_names:
        snake_ids[snake_name] = None

        species_url = search_species_url(snake_name)
        response = requests.get(species_url)
        if response.status_code == 200:
            species = response.json()['results']
            if len(species) > 0:
                snake_ids[snake_name] = species[0]['record']['id']
            else:
                print(f"Error: {snake_name} not found in iNaturalist")
        else:
            print(f"Error {response.status_code} getting {snake_name} from iNaturalist")

            snake_ids

    # save snake_to_species to file
    with open('snake_to_species.json', 'w') as file:
        json.dump(snake_ids, file)

Snake ids file already exists, skipping...


In [33]:
import asyncio
import aiohttp
import aiofiles

async def download_file(session, folder, url, fname):
    try:
        async with session.get(url) as response:
            if response.status == 200:
                fpath = os.path.join(folder, fname)
                async with aiofiles.open(fpath, "wb") as f:
                    await f.write(await response.read())
                    print(f"Downloaded {url} to {fpath}")
            else:
                print(f"Error: {response.status} downloading {url}")
                print(response)
    except aiohttp.ClientResponseError as e:
        print(f"Error: {e.status} downloading {url}")

async def download_all(folder, filenames_and_urls):
    async with aiohttp.ClientSession() as session:
        tasks = [download_file(session, folder, url, filename) for filename, url in filenames_and_urls]
        await asyncio.gather(*tasks)


In [34]:
base_folder_path = os.path.join(curr_dir, "../../data/raw")
if not os.path.exists(base_folder_path):
    raise FileNotFoundError("Base folder path does not exist")

# To avoid downloading images again, set this to False
load_images = True #False

if load_images:
    # Download images for each snake
    for snake_name, snake_id in snake_ids.items():
        if snake_id is None:
            print(f"Skipping {snake_name}, no id found")
            continue
        
        name_without_spaces = snake_name.replace(' ', '_').lower()
        snake_dir = f"{name_without_spaces}-{snake_id}"
        snake_folder_path = os.path.join(base_folder_path, snake_dir)

        if not os.path.exists(snake_folder_path):
            os.mkdir(snake_folder_path)
        else:
            print(f"Skipping {snake_name}, folder already exists")
            continue

        params = {
            'verifiable': 'true',
            'taxon_id': snake_id,
            'order_by': 'votes',
            'quality_grade': 'research',
            'per_page': 1
        }
        
        # Get observations for snake
        response = requests.get(get_observations_url, params=params)

        image_urls = {}

        if response.status_code == 200:
            observations = response.json()['results']
            for o in observations:
                photos = o['photos']
                for p in photos:
                    image_url = p['url'].replace('square.', 'medium.')
                    extension = image_url.split('/')[-1].split('.')[1]
                    filename = f"{p['id']}.{extension}"
                    image_urls[filename] = image_url
        else:
            print(f"Error {response.status_code} getting observations for {snake_name}")

        await download_all(snake_folder_path, image_urls.items()) 
else:
    print("Skipping image download, set load_images to True to download images")



Skipping Carphophis amoenus, folder already exists
Skipping Cemophora coccinea, folder already exists
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/215104356/medium.jpg to /Users/jc/code/aipi540-module1/notebooks/data_sourcing/../../data/raw/coluber_constrictor-27137/215104356.jpg
Downloaded https://inaturalist-open-data.s3.amazonaws.com/photos/215104747/medium.jpg to /Users/jc/code/aipi540-module1/notebooks/data_sourcing/../../data/raw/coluber_constrictor-27137/215104747.jpg
